In [1]:
!pip install duckdb

In [2]:
import duckdb
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from py_duckdb.similarity_join import tokenizers
from py_duckdb.similarity_join import jaccard_join
from py_duckdb.similarity_join import jaccard_join_brute_force
from py_duckdb.similarity_join.default_names import *

In [5]:
_con = duckdb.connect(database=':memory:')

In [6]:
l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
tokenizer = tokenizers.WordsTokzr("'\|'")
# tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'
tokenizer.query()

"select src, rid, len(tks) as rlen, lower(unnest(tks)) as token from ( select src, rid, list_distinct(list_filter(str_split_regex(val, '\\|'), x -> trim(x) != '')) as tks from input ) "

In [7]:
df = pd.read_json("data/test.json", lines=True, orient='records', typ='frame')
df.head()

,realProfileID,date_of_birth,surname,address_1,soc_sec_id,postcode,street_number,suburb,phone_number,state,given_name,age,address_2
0,4606,19450219.0,birmn,etheridb estreet,4842352,2179,39,nedlnyb,07 85883235,qld,mitchelk,39,NaN
1,9317,19450219.0,burman,etheridge street,4842352,2197,39,nedlands,07 85883235,qld,mitchell,39,NaN
2,3521,19810822.0,carmody,rosebery street,4898086,2079,23,deer park,04 42275386,nsw,sarsha,23,NaN
3,4023,19751726.0,carmody,rosebery street,4898886,2006,23,deer park,04 42257386,nsw,sarsjs,23,NaN
4,14,19881101.0,rastar,mccubbib street,9482905,3950,82,milperra,02 84052052,qld,jax,34,summerdale medical centre


In [8]:
_con.execute("drop table if exists data").execute(
    "CREATE TABLE data AS "
    "SELECT realProfileID as rid, concat (date_of_birth, '|', surname, '|', address_1, '|', street_number, '|', postcode, '|',soc_sec_id, '|',"
    "suburb, '|',phone_number, '|', state, '|', given_name, '|',age, '|',address_2 ) as val "
    "FROM df"
).execute("select * from data").fetchall()

[(4606,
  '19450219.0|birmn|etheridb estreet|39|2179|4842352|nedlnyb|07 85883235|qld|mitchelk|39|'),
 (9317,
  '19450219.0|burman|etheridge street|39|2197|4842352|nedlands|07 85883235|qld|mitchell|39|'),
 (3521,
  '19810822.0|carmody|rosebery street|23|2079|4898086|deer park|04 42275386|nsw|sarsha|23|'),
 (4023,
  '19751726.0|carmody|rosebery street|23|2006|4898886|deer park|04 42257386|nsw|sarsjs|23|'),
 (14,
  '19881101.0|rastar|mccubbib street|82|3950|9482905|milperra|02 84052052|qld|jax|34|summerdale medical centre'),
 (3290,
  '19387806.0|dichrera|investigatot street|41|6021|4921302|seven hills|07 37899004|vic|phillip|34|'),
 (7370,
  '|dichivfa|investigarvo street|41|6021|4921302|seven hills|02 31281441|vic|phillip|34|'),
 (6300,
  '19120912.0|bayden|mackerras crescent|60|2511|5842127|glenwood|07 79176904|qld|mildnre|8|'),
 (6859,
  '19120912.0|mildren|mackerras crescent|60|2153|5846577|glenwood||qld|bayden|8|')]

In [9]:
jaccard_join(
    _con, 'data', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
).execute(f"select * from {out_table_name}").fetchall()

[(3290, 7370), (6300, 6859)]

In [10]:
jaccard_join_brute_force(
    _con, 'data', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
).execute(f"select * from {out_table_name}").fetchall()

[(3290, 7370), (6300, 6859)]

In [11]:
l_table = "data"

In [12]:
_con.execute(f"drop table if exists {INPUT_TABLE}").execute(
    f"create table {INPUT_TABLE} as ("
    f"select '{l_table}' as src, {l_key_attr} as rid, {l_join_attr} as val "
    f"from '{l_table}' )"
).execute(f"select * from {INPUT_TABLE}").fetchall()

[('data',
  4606,
  '19450219.0|birmn|etheridb estreet|39|2179|4842352|nedlnyb|07 85883235|qld|mitchelk|39|'),
 ('data',
  9317,
  '19450219.0|burman|etheridge street|39|2197|4842352|nedlands|07 85883235|qld|mitchell|39|'),
 ('data',
  3521,
  '19810822.0|carmody|rosebery street|23|2079|4898086|deer park|04 42275386|nsw|sarsha|23|'),
 ('data',
  4023,
  '19751726.0|carmody|rosebery street|23|2006|4898886|deer park|04 42257386|nsw|sarsjs|23|'),
 ('data',
  14,
  '19881101.0|rastar|mccubbib street|82|3950|9482905|milperra|02 84052052|qld|jax|34|summerdale medical centre'),
 ('data',
  3290,
  '19387806.0|dichrera|investigatot street|41|6021|4921302|seven hills|07 37899004|vic|phillip|34|'),
 ('data',
  7370,
  '|dichivfa|investigarvo street|41|6021|4921302|seven hills|02 31281441|vic|phillip|34|'),
 ('data',
  6300,
  '19120912.0|bayden|mackerras crescent|60|2511|5842127|glenwood|07 79176904|qld|mildnre|8|'),
 ('data',
  6859,
  '19120912.0|mildren|mackerras crescent|60|2153|5846577|glen

In [14]:
_con.execute(f"drop table if exists {TOKENS_VIEW}").execute(
    f"create table {TOKENS_VIEW} as "
    + tokenizer.query()
).execute(f"select * from {TOKENS_VIEW}").fetchall()

[('data', 4606, 10, '19450219.0'),
 ('data', 4606, 10, '39'),
 ('data', 4606, 10, 'birmn'),
 ('data', 4606, 10, 'etheridb estreet'),
 ('data', 4606, 10, '2179'),
 ('data', 4606, 10, '4842352'),
 ('data', 4606, 10, 'nedlnyb'),
 ('data', 4606, 10, '07 85883235'),
 ('data', 4606, 10, 'qld'),
 ('data', 4606, 10, 'mitchelk'),
 ('data', 9317, 10, '19450219.0'),
 ('data', 9317, 10, 'nedlands'),
 ('data', 9317, 10, '07 85883235'),
 ('data', 9317, 10, 'burman'),
 ('data', 9317, 10, 'etheridge street'),
 ('data', 9317, 10, '39'),
 ('data', 9317, 10, '2197'),
 ('data', 9317, 10, '4842352'),
 ('data', 9317, 10, 'qld'),
 ('data', 9317, 10, 'mitchell'),
 ('data', 3521, 10, '19810822.0'),
 ('data', 3521, 10, '4898086'),
 ('data', 3521, 10, '23'),
 ('data', 3521, 10, 'carmody'),
 ('data', 3521, 10, 'rosebery street'),
 ('data', 3521, 10, 'deer park'),
 ('data', 3521, 10, '2079'),
 ('data', 3521, 10, '04 42275386'),
 ('data', 3521, 10, 'nsw'),
 ('data', 3521, 10, 'sarsha'),
 ('data', 4023, 10, '1975172

In [15]:
_con.execute(f"drop table if exists {DOC_FREQ_VIEW}").execute(
    f"CREATE table {DOC_FREQ_VIEW} AS "
    "SELECT token, count(*) AS df "
    f"FROM {TOKENS_VIEW} "
    "GROUP BY token "
).execute(f"drop table if exists {TOKENS_DOC_FREQ_VIEW}").execute(
    f"CREATE table {TOKENS_DOC_FREQ_VIEW} AS "
    f"select rid, rlen, {TOKENS_VIEW}.token, df "
    f", row_number() OVER (PARTITION BY rid ORDER BY df, {TOKENS_VIEW}.token) as pos "
    f"from {TOKENS_VIEW}, {DOC_FREQ_VIEW} "
    f"where {TOKENS_VIEW}.token = {DOC_FREQ_VIEW}.token"
)

In [16]:
_con.execute(f"select * from {TOKENS_DOC_FREQ_VIEW}").fetchall()

[(14, 12, '02 84052052', 1, 1),
 (14, 12, '19881101.0', 1, 2),
 (14, 12, '3950', 1, 3),
 (14, 12, '82', 1, 4),
 (14, 12, '9482905', 1, 5),
 (14, 12, 'jax', 1, 6),
 (14, 12, 'mccubbib street', 1, 7),
 (14, 12, 'milperra', 1, 8),
 (14, 12, 'rastar', 1, 9),
 (14, 12, 'summerdale medical centre', 1, 10),
 (14, 12, '34', 3, 11),
 (14, 12, 'qld', 5, 12),
 (3290, 11, '07 37899004', 1, 1),
 (3290, 11, '19387806.0', 1, 2),
 (3290, 11, 'dichrera', 1, 3),
 (3290, 11, 'investigatot street', 1, 4),
 (3290, 11, '41', 2, 5),
 (3290, 11, '4921302', 2, 6),
 (3290, 11, '6021', 2, 7),
 (3290, 11, 'phillip', 2, 8),
 (3290, 11, 'seven hills', 2, 9),
 (3290, 11, 'vic', 2, 10),
 (3290, 11, '34', 3, 11),
 (3521, 10, '04 42275386', 1, 1),
 (3521, 10, '19810822.0', 1, 2),
 (3521, 10, '2079', 1, 3),
 (3521, 10, '4898086', 1, 4),
 (3521, 10, 'sarsha', 1, 5),
 (3521, 10, '23', 2, 6),
 (3521, 10, 'carmody', 2, 7),
 (3521, 10, 'deer park', 2, 8),
 (3521, 10, 'nsw', 2, 9),
 (3521, 10, 'rosebery street', 2, 10),
 (402

In [17]:
t = threshold

In [18]:
_con.execute(f"drop table if exists {PREFIXES_VIEW}").execute(
    f"create table {PREFIXES_VIEW} as "
    "select rid, rlen, token, pos, df "
    f"from {TOKENS_DOC_FREQ_VIEW} "
    f"where rlen - pos + 1 >= ceil(rlen * {t}) "
).execute(f"select * from {PREFIXES_VIEW}").fetchall()

[(14, 12, '02 84052052', 1, 1),
 (14, 12, '19881101.0', 2, 1),
 (14, 12, '3950', 3, 1),
 (14, 12, '82', 4, 1),
 (14, 12, '9482905', 5, 1),
 (14, 12, 'jax', 6, 1),
 (14, 12, 'mccubbib street', 7, 1),
 (3290, 11, '07 37899004', 1, 1),
 (3290, 11, '19387806.0', 2, 1),
 (3290, 11, 'dichrera', 3, 1),
 (3290, 11, 'investigatot street', 4, 1),
 (3290, 11, '41', 5, 2),
 (3290, 11, '4921302', 6, 2),
 (3521, 10, '04 42275386', 1, 1),
 (3521, 10, '19810822.0', 2, 1),
 (3521, 10, '2079', 3, 1),
 (3521, 10, '4898086', 4, 1),
 (3521, 10, 'sarsha', 5, 1),
 (3521, 10, '23', 6, 2),
 (4023, 10, '04 42257386', 1, 1),
 (4023, 10, '19751726.0', 2, 1),
 (4023, 10, '2006', 3, 1),
 (4023, 10, '4898886', 4, 1),
 (4023, 10, 'sarsjs', 5, 1),
 (4023, 10, '23', 6, 2),
 (4606, 10, '2179', 1, 1),
 (4606, 10, 'birmn', 2, 1),
 (4606, 10, 'etheridb estreet', 3, 1),
 (4606, 10, 'mitchelk', 4, 1),
 (4606, 10, 'nedlnyb', 5, 1),
 (4606, 10, '07 85883235', 6, 2),
 (6300, 11, '07 79176904', 1, 1),
 (6300, 11, '2511', 2, 1),


In [19]:
_con.execute(f"drop table if exists {CANDIDATE_SET_VIEW}").execute(
    f"CREATE table {CANDIDATE_SET_VIEW} AS "
    "SELECT pr1.rid AS rid1, pr2.rid AS rid2 "
    ", MAX(pr1.pos) as maxPos1, MAX(pr2.pos) as maxPos2, count(*) as prOverlap "
    f"FROM {PREFIXES_VIEW} pr1, {PREFIXES_VIEW} pr2 "
    "WHERE pr1.rid < pr2.rid "
    "AND pr1.token = pr2.token "
    # length filter
    f"AND pr1.rlen >= ceil({t} * pr2.rlen)"
    # prefix filter
    # f"AND pr1.rlen - pr1.pos + 1 >= CEIL(pr1.rlen * 2 * {t} / (1+{t})) "
    # positional filter
    "AND LEAST((pr1.rlen - pr1.pos + 1), (pr2.rlen - pr2.pos + 1)) >= "
    f"CEIL((pr1.rlen + pr2.rlen) * {t} / (1 + {t})) "
    "GROUP BY pr1.rid, pr2.rid "
).execute(f"select * from {CANDIDATE_SET_VIEW}").fetchall()

[(3290, 7370, 5, 4, 1), (6300, 6859, 5, 4, 1)]

In [22]:
_con.execute(
    "select r1.rid as rid1, r2.rid as rid2 "
    f", count(*) as overlap "
    f"from {TOKENS_VIEW} as r1, {TOKENS_VIEW} as r2 "
    "where r1.token = r2.token "
    "and r1.rid < r2.rid "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ceil({t} / (1+{t}) * (r1.rlen + r2.rlen))"
).fetchall()

[(3290, 7370, 7), (6300, 6859, 7)]